# EEG analysis

In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys
import tarfile
import gzip
import pandas
import random
import pyunicorn

from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression

from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline

ImportError: No module named numpy

In [43]:
url = 'http://kdd.ics.uci.edu/databases/eeg/'
data_root = '.'
        
def maybe_download(filename, expected_bytes, force=False):
    dest_filename = os.path.join(data_root, filename)
    if force or not os.path.exists(dest_filename):
        print('Attempting to download:', filename) 
        filename, _ = urlretrieve(url + filename, dest_filename)
        print('\nDownload Complete!')
    statinfo = os.stat(dest_filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', dest_filename)
    else:
        raise Exception(
          'Failed to verify ' + dest_filename + '.')
    return dest_filename

maybe_download('eeg_full.tar', 717894656)

Found and verified ./eeg_full.tar


'./eeg_full.tar'

In [44]:
def maybe_extract_all_in_folder(foldername):
    for item in os.listdir(foldername):
        item = os.path.join(foldername, item)
        if os.path.isdir(item):
            print('Going down folder %s' % (item))        
            maybe_extract_all_in_folder(item)
        elif item.endswith('.tar.gz'):
            print('Extracting data for %s' % item)
            tar = tarfile.open(item, "r:gz")
            sys.stdout.flush()
            tar.extractall(foldername)
            tar.close()
            try:
                os.remove(item)
            except Exception as e:
                print('Error when deleting %s: %s' % (item, e.strerror))
        elif item.endswith('.gz'):
            print('Extracting data for %s' % item)
            with gzip.open(item, 'r') as inF:
                outF = open(os.path.splitext(item)[0], 'wb')
                outF.write(inF.read())
            try:
                os.remove(item)
            except Exception as e:
                print('Error when deleting %s: %s' % (item, e.strerror))
        else:
            print('%s already extracted' % (item))
            return
    
data_root = r'/home/miroslav/Source/EEG' 
os.chdir(data_root)
maybe_extract_all_in_folder(os.path.join(data_root, r'eeg_full'))

Going down folder /home/miroslav/Source/EEG/eeg_full/co2c0000383
Extracting data for /home/miroslav/Source/EEG/eeg_full/co2c0000383/co2c0000383.rd.058.gz
Extracting data for /home/miroslav/Source/EEG/eeg_full/co2c0000383/co2c0000383.rd.006.gz
/home/miroslav/Source/EEG/eeg_full/co2c0000383/co2c0000383.rd.074 already extracted
Going down folder /home/miroslav/Source/EEG/eeg_full/co2a0000385
/home/miroslav/Source/EEG/eeg_full/co2a0000385/co2a0000385.rd.029 already extracted
Going down folder /home/miroslav/Source/EEG/eeg_full/co2c0000384
Extracting data for /home/miroslav/Source/EEG/eeg_full/co2c0000384/co2c0000384.rd.111.gz
Extracting data for /home/miroslav/Source/EEG/eeg_full/co2c0000384/co2c0000384.rd.091.gz
/home/miroslav/Source/EEG/eeg_full/co2c0000384/co2c0000384.rd.038 already extracted
Going down folder /home/miroslav/Source/EEG/eeg_full/co2c0000359
Extracting data for /home/miroslav/Source/EEG/eeg_full/co2c0000359/co2c0000359.rd.023.gz
/home/miroslav/Source/EEG/eeg_full/co2c0000

/home/miroslav/Source/EEG/eeg_full/co2a0000394/co2a0000394.rd.026 already extracted
Going down folder /home/miroslav/Source/EEG/eeg_full/co2a0000390
/home/miroslav/Source/EEG/eeg_full/co2a0000390/co2a0000390.rd.025 already extracted
Going down folder /home/miroslav/Source/EEG/eeg_full/co2c0000351
/home/miroslav/Source/EEG/eeg_full/co2c0000351/co2c0000351.rd.061 already extracted
Going down folder /home/miroslav/Source/EEG/eeg_full/co2a0000443
/home/miroslav/Source/EEG/eeg_full/co2a0000443/co2a0000443.rd.002 already extracted
Going down folder /home/miroslav/Source/EEG/eeg_full/co2a0000419
/home/miroslav/Source/EEG/eeg_full/co2a0000419/co2a0000419.rd.109 already extracted
Going down folder /home/miroslav/Source/EEG/eeg_full/co2a0000369
/home/miroslav/Source/EEG/eeg_full/co2a0000369/co2a0000369.rd.097 already extracted
Going down folder /home/miroslav/Source/EEG/eeg_full/co2a0000384
/home/miroslav/Source/EEG/eeg_full/co2a0000384/co2a0000384.rd.117 already extracted
Going down folder /hom

In [54]:
# Extract the time series
def extract_random_data_series(folder):
    selected_subfolder = \
        random.choice([os.path.join(folder, x) for x in os.listdir(folder) if os.path.isdir(os.path.join(folder, x))])
    print('Selected subfolder: %s' % (selected_subfolder))
    selected_file = \
        random.choice([os.path.join(selected_subfolder, x) for x in os.listdir(selected_subfolder) if 
                       os.path.isfile(os.path.join(selected_subfolder, x)) and not x.endswith(".gz")])
    df = pd.read_table(selected_file, skiprows=5, names=['chan', 'ind', 'val'], delim_whitespace=True, usecols=range(1,4))
    df = df.loc[df['chan'] == 'FP1']
    return df['val']

random_ts = extract_random_data_series(r'./eeg_full')
random_ts

Selected subfolder: ./eeg_full/co2a0000414


0     -1.648
1      0.793
2      2.258
3      2.747
4      2.258
5      1.770
6      1.282
7      0.793
8      0.305
9     -0.671
10    -2.136
11    -3.113
12    -3.601
13    -3.601
14    -3.601
15    -3.601
16    -3.601
17    -3.601
18    -4.089
19    -4.578
20    -5.554
21    -5.554
22    -5.066
23    -3.601
24    -2.625
25    -2.136
26    -3.113
27    -5.066
28    -6.042
29    -6.042
       ...  
226    2.258
227    2.747
228    2.258
229    0.793
230   -0.183
231   -1.160
232   -1.648
233   -1.160
234   -1.160
235   -2.136
236   -4.089
237   -6.042
238   -7.019
239   -5.066
240   -2.136
241    0.305
242    0.793
243   -1.160
244   -5.066
245   -7.996
246   -7.996
247   -5.554
248   -2.625
249   -0.671
250   -1.160
251   -3.113
252   -5.066
253   -6.042
254   -6.042
255   -5.554
Name: val, Length: 256, dtype: float64

In [ ]:
def display_rp(ts):
    